# Tema 1: Herramientas python para ciencia de los datos

## 1.2 Introducción a pandas

Los ejemplos y la presentación que sigue está tomada del capítulo 4 del libro:

[*Python for Data Analysis*](https://wesmckinney.com/pages/book.html)  
**Wes McKinney**  
O'Reilly 2017

Github con el material del libro: [Github](https://github.com/wesm/pydata-book)

In [1]:
import pandas as pd

In [2]:
#from pandas import Series, DataFrame

In [3]:
import numpy as np
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

### *Series* de pandas: introducción

Una *Serie* en pandas es una estructura tipo array unidimensional, junto con una secuencia (su *índice*) de "etiquetaa" que "nombran" a los elementos del array. El caso más simple: 

In [4]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

En el ejemplo, el índice es la columna de la izquierda (por defecto se ha creado un índice numérico) y los valores de la serie son la columna de la derecha.

Con los atributos `values` e `index` accedemos al array de valores y al índice:

In [5]:
obj.values

array([ 4,  7, -5,  3])

In [6]:
obj.index  # es similar a range(4), pero de una clase `RangeIndex`, específica pandas

RangeIndex(start=0, stop=4, step=1)

Podemos crear una serie proporcionando explícitamente el índice:

In [7]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [8]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

El índice en una serie de pandas, generaliza la nocion de índice en numpy. Podemos acceder y modificar los valores de una serie a través del índice, 

In [9]:
obj2['a']

-5

In [10]:
obj2['d'] = 6
obj2

d    6
b    7
a   -5
c    3
dtype: int64

Podemos incluso obtener una subserie, mediante un subconjunto reordenado de sus índices:

In [11]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

También muchas de las funcionalidades de los arrays de numpy, como por ejemplo el indexado booleano o las operaciones vectorizadas, se aplican también a las series:

In [12]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [13]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [14]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

Una manera muy útil de pensar en una serie de pandas es como un diccionario (en el que el índice son las claves) de longitud fija y en el que el orden es significativo. Por ejemplo, para consultar si una etiqueta está en un índice de una serie, podemos usar `in`, como en los diccionarios:

In [15]:
'b' in obj2

True

In [16]:
'e' in obj2

False

De hecho, una manera muy frecuente de crear una serie es a partir de un diccionario. Las claves se ordenarán y formarán el índice de la serie, como en el siguiente ejemplo:

In [17]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

Si queremos introducir un orden específico entre las claves del diccionario, entonces podemos combinar el pasar el diccionario junto con la lista de etiquetas ordenadas:

In [18]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

Nótese que solo se incluye en el índice lo incluido en la lista (por ejemplo `Utah` no forma parte del índice a pesar de que es una clave del diccionario). Como `California` no es una clave del diccionario, pero se ha incluido en el índice, se incluye con valor `NaN`(*Not a Number*), que es la manera en pandas para indicar valores inexistentes. 

Con `isnull` podemos localizar qué entradas de la serie tienen valores inexistentes:

In [19]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [20]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

En las series, como con los arrays de numpy, podemos realizar operaciones vectorizadas. Lo interesante aquí es que las operaciones se *alinean* por las correspondientes etiquetas. Por ejemplo:

In [21]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [22]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [23]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Como se observa, se han sumado los correspondientes valores en aquellas entradas en los que el valor estaba disponible. En el caso de California, la operación no se ha podido realizar ya que en una de las series no estaba (y además en la otra estaba con valor `NaN`). En el caso de Utah, sólo estaba en una de las series.

Tanto la serie como el índice de la serie tienen un atributo nombre, del cual veremos más adelante su utilidad:

In [24]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

In [25]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

Por último, podemos modificar (destructivamente) un índice mediante la correspondiente asignación al atributo `index`:

In [26]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### *DataFrame* de pandas: introducción

Un *DataFrame* de pandas es una tabla bidimensional, con las columnas y las filas en un determinado orden. Cada columna puede ser de un tipo diferente. En términos de índices: tanto las filas como las columnas están indexadas. Puede ser visto como un diccionario en el que las claves son las etiquetas de las columnas, y todos los valores son *Series* de pandas que comparten el mismo índice. 

Aunque hay muchas maneras de crear un *DataFrame*, una de las más frecuentes es mediante un diccionario cuyos valores asociados a las claves son listas de la misma longitud. Por ejemplo:   

In [27]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

Nótese la forma en que se muestra un *DataFrame* en Jupyter:

In [28]:
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


Cuando los *DataFrames* son grandes, puede ser útil el método `heads`, que muestar solo las primeras cinco filas de la tabla:

In [29]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


Como en el caso de las Series, podemos proporcionar las columnas en un orden determinado: 

In [30]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


Y también podemos indicar expresamente el índice de las filas:

In [31]:
pd.DataFrame(data, columns=['pop', 'year', 'state'], index=['one', 'two', 'three', 'four',
                             'five', 'six'])

,pop,year,state
one,1.5,2000,Ohio
two,1.7,2001,Ohio
three,3.6,2002,Ohio
four,2.4,2001,Nevada
five,2.9,2002,Nevada
six,3.2,2003,Nevada


Si al proporcionar los nombres de las columnas damos una de ellas que no aparece en el diccionario con los datos, entonces se crea la columnos con valores no determinados:

In [32]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four',
                             'five', 'six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


Los atributos `index` y `columns` nos devuelven los correspondientes índices de las filas y de las columnas (ambos son objetos `Index` de pandas): 

In [33]:
frame2.index

Index(['one', 'two', 'three', 'four', 'five', 'six'], dtype='object')

In [34]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

Para acceder a una columna en concreto del *DataFrame*, podemos hacerlo usando la notación de diccionario, o también como atributo. En ambos casos se devuelve la correspondiente columna como un objeto *Series* de pandas: 

In [35]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [36]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

De igual manera, podemos acceder a una fila del *DataFrame* mediante el atributo `loc`. La fila también se devuelve como un objeto *Series*, cuyo índice está formado por los nombres de las columnas:

In [37]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

Veamos ahora ejemplos sobre cómo podemos modificar columnas mediante asignaciones. En general, muchas de los procedimientos de numpy aquí también son válidos, pero teniendo en cuenta que indexamos mediante el nombre de la columna: 

Por ejemplo, asignar el mismo valor a toda una columna:

In [38]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


O asignar mediante una secuencia:

In [39]:
frame2['debt'] = np.arange(6.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


Cuando a una columna le asignamos una lista o un array, como en el ejemplo anterior, la longitud de la secuencia debe de coincidir con el número de filas del *DataFrame*. Sin embargo, podemos asignar con un objeto *Series* y los valores se asignarán alineando por el valor del índice, incluso parcialmente (al resto se el asignará *NaN*):

In [40]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


Si asignamos una columna que no existe, ésta se creará. Por ejemplo:

In [41]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


Con `del` borranos una columna completa:

In [42]:
del frame2['eastern']
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

In [43]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


Otra forma de crear un *DataFrame* es a partir de un diccionario de diccionarios, en el que las claves externas constituyen las etiquetas de las columnas, y las internas como las de las filas:

In [44]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

In [45]:
frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


Como en numpy, podemos también obtener la traspuesta de un *DataFrame*, quedando las filas como columns y viceversa:

In [46]:
frame3.T

,2001,2002,2000
Nevada,2.4,2.9,NaN
Ohio,1.7,3.6,1.5


Si usamos un diccionario anidado, y le damos explícitamente el índice como una secuencia, entonces las filas se tienen en cuenta y se ordenan como se indica en la secuencia. Lo mismo ocurre si se le especifican explícitamente las columnas:

In [47]:
pd.DataFrame(pop, index=[2001, 2002, 2003],columns=["Ohio","California","Nevada"])

,Ohio,California,Nevada
2001,1.7,NaN,2.4
2002,3.6,NaN,2.9
2003,NaN,NaN,NaN


También se puede dar un *DataFrame* como un diccionario en el que cada clave (columna) tiene asociada una serie: 

In [48]:
pdata = {'Ohio': frame3['Ohio'][:-1],
         'Nevada': frame3['Nevada'][:2]}
pd.DataFrame(pdata)

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9


Con el atributo `name` (tanto de `index`como de `columns`) podemos acceder y/o modificar el nombre de las filas y las columnas, que se mostrarán al mostrarse la tabla:

In [49]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


Por último, mediante `values`, accedemos a un array bidimensional con los valores de cada entrada de la tabla:

In [50]:
frame3.values

array([[2.4, 1.7],
       [2.9, 3.6],
       [nan, 1.5]])

In [51]:
frame2.values # el dtype se acomoda a lo más general. 

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

#### Resumen de algunas maneras de crear un *DataFrame*:

* Array bidimensional, opcionalmente con `index`y/o `column`
* Diccionario de arrays, listas o tuplas de la misma longitud; cada clave se refiere a una columna
* Diccionario de *Series*; cada clave es una columna y las filas se alinean según los índices de las series, o bien se le pasa explícitamente el índice. 
* Diccionario de diccionarios: las claves externas son las columnas, las internas las filas.
* Lista de listas o tuplas: como en el caso de array bidimensional. 

### Funcionalidades básicas

#### Reindexado

Mediante reindexado, creamos un nuevo objeto adaptándolo a un nuevo índice. Veámos lo con un ejemplo:

In [52]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

Reindexamos con `reindex`. Nótese cómo la nueva etiqueta `'e'` se incluye en la nueva serie, con valor no especificado: 

In [53]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

Se puede reindexar un *DataFrame*, ajustando tanto las filas como las columnas. Veámoslo con un ejemplo:

In [54]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


Si a `reindex`se le pasa solamente una secuencia, entonces se entiende que se quieren reajustar las filas:

In [55]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


Para reindexar las columnas, hay que pasar el argumento clave `columns`:

In [56]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


#### Eliminando entradas de un eje

Ya vimos antes que `del` borra una columna completa de un *DataFrame*. Mediante `drop`, podemos *crear nuevos objetos* resultantes de eliminar filas o columnas completas. Veamos algunos ejemplos. En primer lugar, con las *Series*:

In [57]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [58]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [59]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

Ahora veamos el uso de `drop` con *DataFrames*:

In [60]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Por defecto, se eliminan del eje 0 (las filas):

In [61]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


Podemos eliminar columnas, indicándo que se quiere hacer en `axis=1` o `axis='columns'`:

In [62]:
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [63]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


Como hemos dicho, por defecto, `drop` devuelve un nuevo objeto. Pero como otras funciones, podrían actuar de manera destructiva, modificando el objeto original. Para ello, hay que indicarlo con el argumento clave `inplace`:

In [64]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

### Indexado, selección y filtrado

El acceso a los elementos de un objeto *Series* se hace de manera similar a los arrays de numpy, excepto que también podemos usar el correspondiente valor del índice, a demás de la posición numérica. Veámoslo con un ejemplo: 

In [65]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

Podemos acceder al segundo elemento de la serie anterir, bien mediante el valor `'b'`, o por la posición 1, ambos accesos son equivalentes:

In [66]:
obj['b'],obj[1]

(1.0, 1.0)

Más ejemplos de indexado en objetos de tipo *Series*:

In [67]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [68]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [69]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [70]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

Podemos hacer también *slicing* con las etiquetas de un índice. Existe una diferencia importante, y es que el límite superior se considera incluido:

In [71]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

Podemos incluso establecer valores usando *slicing*, como en los arrays de numpy:

In [72]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

Para *DataFrames*, el acceso mediante una etiqueta, extrae por defecto la correspondiente columna en forma de Series, como ya habíamos visto anteriormente. En el siguiente ejemplo: 

In [73]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [74]:
data['two'] ### 

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

También se admite indexado mediante una lista de etiquetas:

In [75]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


Hay un par de casos particulares, que no funciona seleccionando columnas: si hacemos slicing con enteros, nos estamos refiriendo a las filas:

In [76]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


También el indexado booleano filtar por filas:

In [77]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### Selección con loc e iloc

Con `loc` tenemos una manera de acceder a los datos de un *DataFrame* de una manera similar a los array de numpy, usando las etiquetas de las filas y columnas. Con `iloc` podemos usar índices enteros, como con numpy. Veamos algunos ejemplos.

In [78]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Para acceder a la fila etiquetada como `Colorado` y sólo a las columnas `two` y `three`, en ese orden (nótese que se devuelve una serie):

In [79]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int64

Un ejemplo, similar, pero ahora con índices numéricos. La fila de índice 2, sólo con las columnas 3, 0 y 1. 

In [80]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int64

La fila de índice 2:

In [81]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int64

Podemos especificar una subtabla por sus filas y columnas

In [82]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,4,5
Utah,11,8,9


Podemos usar slicing con las etiquetas (recordar que el límite superior es inclusive):

In [83]:
data.loc[:'Utah', 'two']

Ohio        1
Colorado    5
Utah        9
Name: two, dtype: int64

Un ejemplo algo más complicado. Seleccionamos primero las tres primeras columnas mediante slicing con enteros, y luego seleccionamos las filas que en la columna etiquetada con `'three'` tienen un valor mayor que 5:

In [84]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,4,5,6
Utah,8,9,10
New York,12,13,14


### Operaciones aritméticas con alineamiento de índices

Aunque ya lo hemos visto anteriormente, hagamos de nuevo hincapié en que al hacer operaciones aritméticas entre objetos, estas se hacen alineando las correspondientes eetiquetas de los índices (y dejando NaN cuando alguno de los operandos no está definido):

Veamos un ejemplo con *Series*:

In [85]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])

In [86]:
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [87]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [88]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

En el caso de los *DataFrames*, el alineamiento se produce en las filas y columnas:

In [89]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [90]:
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [91]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [92]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


#### Operaciones aritméticas con valores de relleno

In [93]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list('abcde'))

df1.loc[1, 'b'] = np.nan
df2.loc[1, 'b'] = np.nan

In [94]:
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,NaN,6.0,7.0
2,8.0,9.0,10.0,11.0


In [95]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


Al sumar ambos *DataFrames*,téngase en cuanta que la fila 3 no existe en una de ellas, al igual que la columna `'e'`. POr tanto, en el resultado, se crearán en esas fila y columna con valores no determinados: 

In [96]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


Sin embargo, podemos usar el método `add` con "valor de relleno" 0, y en ese caso, cuando uno de los operandos no esté definido, se tome ese valor por defecto (cero en este caso). Nótese que si ninguno de los operandos está definido (como en `(1,'b')`), entonces no se aplica el relleno. 

In [97]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,NaN,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


Como `add`, existem otras operaciones aritméticas que permiten `fill_value`: `sub`, `mul`, `div`, `pow`, ...

Relacionado con esto, también es interesante destacar que cuando se reindexa un objeto, podemos especificar el valor de relleno, cuando el valor no esté especificado en el objeto original:

In [98]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,NaN,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


### Aplicación de funciones a las "vectorizadas"

Como en numpy, podemos aplicar funciones de forma vectorizada a todos los valores de un *Series* o un *DataFrame*:

In [99]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,-0.204708,0.478943,-0.519439
Ohio,-0.555730,1.965781,1.393406
Texas,0.092908,0.281746,0.769023
Oregon,1.246435,1.007189,-1.296221


Por ejemplo, aplicamos valor absoluto a cada elemento de la tabla:

In [100]:
np.abs(frame)

,b,d,e
Utah,0.204708,0.478943,0.519439
Ohio,0.555730,1.965781,1.393406
Texas,0.092908,0.281746,0.769023
Oregon,1.246435,1.007189,1.296221


Otra operación muy frecuente consiste en aplicar una función definida sobre arrays unidimensionales, a lo largo de uno de los ejes (filas o columnas). Esto se hace con el método `apply`de los *DataFrames*.

In [101]:
f = lambda x: x.max() - x.min()
frame.apply(f)

b    1.802165
d    1.684034
e    2.689627
dtype: float64

El eje por defecto para hacer un `apply`es el 0, es decir el de las filas (y por tanto aplica la opración sobre cada columna). Podemos usar el argumento `axis` para especificar que queremos aplicar en el sentido de las columnas (y por tanto, hacer el cálculo sobre las filas): 

In [102]:
frame.apply(f, axis='columns')

Utah      0.998382
Ohio      2.521511
Texas     0.676115
Oregon    2.542656
dtype: float64

En realidad, hay muchos funciones estadísticas (como `sum` o `mean`) que de por sí ya están adaptadas a *DataFrames* y no necesitan usar apply, como veremos más adelante. 

### Ordenación

En pandas tenemos posibilidad de ordenar bien teniendo en cuenta las etiquetas de las filas o columnas, o bien con los valores propiamente dichos:

Comenzamos con un ejemplo con *Series*:

In [103]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj

d    0
a    1
b    2
c    3
dtype: int64

Con `sort_index` ordenamos la serie por las etiquetas del índice:

In [104]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

Con *DataFrame*, podemos ordenar por las etiquetas de las filas, o también por las columnas, usando el argumento `axis`:

In [105]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [106]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [107]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [108]:
frame.sort_index(axis=1, ascending=False) # se puede especificar si es ascendente o descendente

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


Con `sort_values`, ordenamos por los valores de las entradas. Por ejemplo, en una serie:

In [109]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

Si ordenamos por valores, los *NaN* se sitúan al final:

In [110]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

Con `sort_values` aplicado a *DataFrames*, podemos ordenar por el valor de alguna columna, o incluso por el valor de una fila, usando el argumento clave `'by'`:

In [111]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]},index=['j','k','l','m'])
frame

,b,a
j,4,0
k,7,1
l,-3,0
m,2,1


Ordenación de la tabla según la columna `'b'`:

In [112]:
frame.sort_values(by='b')

,b,a
l,-3,0
m,2,1
j,4,0
k,7,1


Por el valor de dos columnas (lexicográficamente):

In [113]:
frame.sort_values(by=['a', 'b'])

,b,a
l,-3,0
j,4,0
m,2,1
k,7,1


Por el valor de una fila:

In [114]:
frame.sort_values(axis=1,by='k')

,a,b
j,0,4
k,1,7
l,0,-3
m,1,2


### Funciones estadísticas descriptivas

Los objetos de pandas incorporan una serie de métodos estadísticos que calculan un valor a partir de los valores de una serie o de filas o columnas de un *DataFrame*. Una particularidad interesante es que manejan adecuadamente los valores no especificados. Veamos algunos ejemplos:

In [115]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


Por defecto, el método `sum` calcula la suma de cada columna de un *DataFrame*. Los valores NaN se tratan como 0 (a no ser que toda la serie sea de valores NaN):

In [116]:
df.sum()

one    9.25
two   -5.80
dtype: float64

Como es habitual, con el parámetro `axis` podemos hacerlo por el eje de las columnas:

In [117]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

Con `mean` calculamos la media de filas o columnas según el eje elegido con *axis*. El parámetro `skipna` nos permite indicar si se excluyen o no los valores NaN:

In [118]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

El método `idxmax`nos da la etiqueta donde se alcanza el mínimo de cada columna (o cada fila)

In [119]:
df.idxmax()

one    b
two    d
dtype: object

El método `cumsum` nos da los acumulados por fila o por columna:

In [120]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


Por último el método `describe` produce un resumen con las estadísticas más importantes:

In [121]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


In [122]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object